In [1]:
from pathlib import Path
from dask.distributed import Client, LocalCluster, as_completed
import dask.dataframe as dd
import pandas as pd
import pickle
from tqdm.auto import tqdm

# Prepare a dask cluster and client
cluster = LocalCluster(n_workers=12, threads_per_worker=1)
client = Client(cluster)
client

/home/cianh/Programming/Git_Projects/Aconity_ML_Expt1/.venv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 42341 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42341/status,
Dashboard: http://127.0.0.1:42341/status,Workers: 12
Total threads: 12,Total memory: 15.27 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41583,Workers: 12
Dashboard: http://127.0.0.1:42341/status,Total threads: 12
Started: Just now,Total memory: 15.27 GiB
Comm: tcp://127.0.0.1:36129,Total threads: 1
Dashboard: http://127.0.0.1:45385/status,Memory: 1.27 GiB
Nanny: tcp://127.0.0.1:34663,


In [2]:
# Load X and y for training
samples = list(range(1, 82))

X = {i: dd.read_parquet(f"sample_X/{i}") for i in samples}

with open("sample_y.pkl", "rb") as f:
    y = pickle.load(f)

In [8]:
def separate_params(d):
    for sample, df in d.items():
        params = df[
            [
                "Laser power, P (W)",
                "Scan speed, V (mm/sec)",
                "Spot size, F (µm)",
                "Hatch spacing, H (µm)",
            ]
        ]
        params = params.rename(
            columns={
                "Laser power, P (W)": "laser_power",
                "Scan speed, V (mm/sec)": "scan_speed",
                "Spot size, F (µm)": "spot_size",
                "Hatch spacing, H (µm)": "hatch_spacing",
            }
        )
        yield sample, params


params = dict(separate_params(y))

In [10]:
def separate_composition(d):
    for sample, df in d.items():
        df = df[["Ni (Norm)", "Ti (Norm)"]]
        df = df.rename(columns={"Ni (Norm)": "Ni", "Ti (Norm)": "Ti"})
        yield sample, df


composition = dict(separate_composition(y))

In [135]:
with open("sample_X.pkl", "wb") as f:
    pickle.dump(
        {
            int(sample.stem): (
                params[int(sample.stem)],
                dd.read_parquet(sample),
            )
            for sample in Path("sample_X").iterdir()
        },
        f,
    )

with open("sample_y.pkl", "wb") as f:
    pickle.dump(composition, f)

In [100]:
def renormalize_pyrometry(df):
    # temps are usually centered around roughly 1000mV
    df["t"] = (df["t"] / 1000.0) - 1  # adjust to be between -1 and 1 is most cases
    df["x"] = df["x"] / 10.0  # Conversion to cm brings to closer to normalized range
    df["y"] = df["y"] / 10.0
    df["z"] = df["z"] / 10.0
    df["time"] = df["time"] / (60 * 60)  # convert to hours
    return df


futures = [
    renormalize_pyrometry(dd.read_parquet(sample)).to_parquet(
        Path("sample_X_new") / sample.name,
        compression="lz4",
        write_metadata_file=True,
        compute=False,
    )
    for sample in Path("sample_X").iterdir()
]

for i in tqdm(list(range(0, len(futures), 12))):
    dd.compute(*futures[i : i + 12])

  0%|          | 0/7 [00:00<?, ?it/s]